# Tensorflow Experimentation Setup

In the previous tutorial we introduced the tensorflow framework and some very basic functionality that it can provide. In this tutorial we will present a far more readable and research oriented tensorflow based code-base that allows one to quickly build new model architectures and research experiments in tensorflow. The proposed code-structure has been tested in real research and has proven a very readable and easily modifiable setup. 

The tf_mlp package contains all necessary modules with which one can easily train and evaluate a classifier. The following packages can be found:

    1. utils: 
        1. network_summary: Provides utilities with which one can get network summaries, such as the number of parameters and names of layers.
        2. parser_utils which are used to parse arguments passed to the training scripts.
      3. storage, which is responsible for storing network statistics.
    2. data_providers.py : Provides the data providers for training, validation and testing.
    3. network_architectures.py: Defines the network architectures. We provide VGGNet as an example.
    4. network_builder.py: Builds the tensorflow computation graph. In more detail, it builds the losses, tensorflow summaries and training operations.
    5. network_trainer.py: Runs an experiment, composed of training, validation and testing. It is setup to use arguments such that one can easily write multiple bash scripts with different hyperparameters and run experiments very quickly with minimal code changes.


To run an experiment just run:

```
python network_trainer.py --batch_size 128 --epochs 100 --experiment_prefix VGG_EMNIST --tensorboard_use True --batch_norm_use True --strided_dim_reduction True --seed 16122017
```


The arguments after network_trainer.py can be changed to suit your experimental needs. For more arguments and exploring how to add new arguments of your own please view parser_utils.py under utils and network_trainer.py as they provide all the functionality that is necessary to add arguments.

Finally remember to make sure your code is not just efficient but readable. Research code has a very bad reputation, so let's try to improve readability one research line code at a time

To run tensorboard just point tensorboard to the correct logs repository as follows:
    ```tensorboard --port 60xx --logdir /path/to/logs```

In [1]:
import tensorflow as tf
inputs = tf.placeholder(tf.float32, [None, 3000], 'inputs')
targets = tf.placeholder(tf.float32, [None, 10], 'targets')

/afs/inf.ed.ac.uk/user/s17/s1773005/miniconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [95]:
# 2 hidden layers GD learning rate 10-3
n_hidden_1 = 100
n_hidden_2 = 100
beta = 0.01 #0.009

weights = {
    'h1': tf.Variable(tf.random_normal([3000, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, 10]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([10]))
}
def multilayer_2hidden100(data):
    layer_1 = tf.nn.relu(tf.matmul(data, weights['h1']) + biases['b1'])
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights['h2']) + biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer
out_layer = multilayer_2hidden100(inputs)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=targets)
error = tf.reduce_mean(per_datapoint_errors)
#regularizer = tf.nn.l2_loss(weights)
#error = tf.reduce_mean(loss + beta * regularization)

per_datapoint_pred_is_correct = tf.equal(tf.argmax(out_layer, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(error)
sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)
import data_providers as data_providers
train_data = data_providers.MSD10GenreDataProvider('train', batch_size=50, flatten=True, one_hot=True)
valid_data = data_providers.MSD10GenreDataProvider('valid', batch_size=50, flatten=True, one_hot=True)
num_epoch = 20
for e in range(num_epoch):
    running_error = 0.
    running_acc = 0.
    for input_batch, target_batch in train_data:
        _, batch_error, batch_acc = sess.run(
            [train_step, error, accuracy], 
            feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
        running_acc += batch_acc
    running_error /= train_data.num_batches
    running_acc /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e + 1, running_error), 
          'running acc average = {1:.2f}'.format(e + 1, running_acc))

End of epoch 1: running error average = 509.86 running acc average = 0.20
End of epoch 2: running error average = 54.51 running acc average = 0.18
End of epoch 3: running error average = 21.51 running acc average = 0.17
End of epoch 4: running error average = 12.92 running acc average = 0.16
End of epoch 5: running error average = 9.64 running acc average = 0.15
End of epoch 6: running error average = 7.53 running acc average = 0.16
End of epoch 7: running error average = 6.30 running acc average = 0.15
End of epoch 8: running error average = 5.44 running acc average = 0.15
End of epoch 9: running error average = 5.00 running acc average = 0.15
End of epoch 10: running error average = 4.48 running acc average = 0.15
End of epoch 11: running error average = 4.21 running acc average = 0.15
End of epoch 12: running error average = 4.00 running acc average = 0.15
End of epoch 13: running error average = 3.85 running acc average = 0.15
End of epoch 14: running error average = 3.74 running a

In [96]:
# 2 hidden layers GD learning rate 10-4
n_hidden_1 = 100
n_hidden_2 = 100
beta = 0.01 #0.009

weights = {
    'h1': tf.Variable(tf.random_normal([3000, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, 10]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([10]))
}
def multilayer_2hidden100(data):
    layer_1 = tf.nn.relu(tf.matmul(data, weights['h1']) + biases['b1'])
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights['h2']) + biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer
out_layer = multilayer_2hidden100(inputs)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=targets)
error = tf.reduce_mean(per_datapoint_errors)
#regularizer = tf.nn.l2_loss(weights)
#error = tf.reduce_mean(loss + beta * regularization)

per_datapoint_pred_is_correct = tf.equal(tf.argmax(out_layer, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate=0.0001).minimize(error)
sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)
import data_providers as data_providers
train_data = data_providers.MSD10GenreDataProvider('train', batch_size=50, flatten=True, one_hot=True)
valid_data = data_providers.MSD10GenreDataProvider('valid', batch_size=50, flatten=True, one_hot=True)
num_epoch = 20
for e in range(num_epoch):
    running_error = 0.
    running_acc = 0.
    for input_batch, target_batch in train_data:
        _, batch_error, batch_acc = sess.run(
            [train_step, error, accuracy], 
            feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
        running_acc += batch_acc
    running_error /= train_data.num_batches
    running_acc /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e + 1, running_error), 
          'running acc average = {1:.2f}'.format(e + 1, running_acc))

End of epoch 1: running error average = 1764.26 running acc average = 0.15
End of epoch 2: running error average = 924.03 running acc average = 0.20
End of epoch 3: running error average = 646.06 running acc average = 0.21
End of epoch 4: running error average = 471.24 running acc average = 0.21
End of epoch 5: running error average = 352.70 running acc average = 0.21
End of epoch 6: running error average = 261.04 running acc average = 0.21
End of epoch 7: running error average = 194.71 running acc average = 0.21
End of epoch 8: running error average = 144.15 running acc average = 0.21
End of epoch 9: running error average = 106.78 running acc average = 0.21
End of epoch 10: running error average = 81.91 running acc average = 0.20
End of epoch 11: running error average = 63.46 running acc average = 0.21
End of epoch 12: running error average = 50.38 running acc average = 0.20
End of epoch 13: running error average = 42.30 running acc average = 0.20
End of epoch 14: running error averag

In [98]:
# 2 hidden layers GD learning rate 10-5
n_hidden_1 = 100
n_hidden_2 = 100
beta = 0.01 #0.009

weights = {
    'h1': tf.Variable(tf.random_normal([3000, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, 10]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([10]))
}
def multilayer_2hidden100(data):
    layer_1 = tf.nn.relu(tf.matmul(data, weights['h1']) + biases['b1'])
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights['h2']) + biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer
out_layer = multilayer_2hidden100(inputs)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=targets)
error = tf.reduce_mean(per_datapoint_errors)
#regularizer = tf.nn.l2_loss(weights)
#error = tf.reduce_mean(loss + beta * regularization)

per_datapoint_pred_is_correct = tf.equal(tf.argmax(out_layer, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate=0.00001).minimize(error)
sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)
import data_providers as data_providers
train_data = data_providers.MSD10GenreDataProvider('train', batch_size=50, flatten=True, one_hot=True)
valid_data = data_providers.MSD10GenreDataProvider('valid', batch_size=50, flatten=True, one_hot=True)
num_epoch = 50
for e in range(num_epoch):
    running_error = 0.
    running_acc = 0.
    for input_batch, target_batch in train_data:
        _, batch_error, batch_acc = sess.run(
            [train_step, error, accuracy], 
            feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
        running_acc += batch_acc
    running_error /= train_data.num_batches
    running_acc /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e + 1, running_error), 
          'running acc average = {1:.2f}'.format(e + 1, running_acc))

End of epoch 1: running error average = 3307.33 running acc average = 0.11
End of epoch 2: running error average = 2221.83 running acc average = 0.12
End of epoch 3: running error average = 1954.74 running acc average = 0.14
End of epoch 4: running error average = 1778.21 running acc average = 0.15
End of epoch 5: running error average = 1653.09 running acc average = 0.15
End of epoch 6: running error average = 1523.82 running acc average = 0.16
End of epoch 7: running error average = 1420.52 running acc average = 0.17
End of epoch 8: running error average = 1333.63 running acc average = 0.17
End of epoch 9: running error average = 1244.29 running acc average = 0.17
End of epoch 10: running error average = 1182.66 running acc average = 0.18
End of epoch 11: running error average = 1122.90 running acc average = 0.18
End of epoch 12: running error average = 1065.90 running acc average = 0.19
End of epoch 13: running error average = 1008.14 running acc average = 0.19
End of epoch 14: runn

In [5]:
# 100 hidden units learning rate 10-4
n_hidden_1 = 100
n_hidden_2 = 100
n_hidden_3 = 100
beta = 0.01 #0.009

weights = {
    'h1': tf.Variable(tf.random_normal([3000, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, 10]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([10]))
}
def multilayer_3hidden100(data):
    layer_1 = tf.nn.relu(tf.matmul(data, weights['h1']) + biases['b1'])
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights['h2']) + biases['b2'])
    layer_3 = tf.nn.relu(tf.matmul(layer_2, weights['h3']) + biases['b3'])
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer
out_layer = multilayer_3hidden100(inputs)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=targets)
error = tf.reduce_mean(per_datapoint_errors)
#regularizer = tf.nn.l2_loss(weights)
#error = tf.reduce_mean(loss + beta * regularization)

per_datapoint_pred_is_correct = tf.equal(tf.argmax(out_layer, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate=0.0001).minimize(error)
sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)
import data_providers as data_providers
train_data = data_providers.MSD10GenreDataProvider('train', batch_size=50, flatten=True, one_hot=True)
valid_data = data_providers.MSD10GenreDataProvider('valid', batch_size=50, flatten=True, one_hot=True)
num_epoch = 10
for e in range(num_epoch):
    running_error = 0.
    running_acc = 0.
    for input_batch, target_batch in train_data:
        _, batch_error, batch_acc = sess.run(
            [train_step, error, accuracy], 
            feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
        running_acc += batch_acc
    running_error /= train_data.num_batches
    running_acc /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e + 1, running_error), 
          'running acc average = {1:.2f}'.format(e + 1, running_acc))

End of epoch 1: running error average = 3106.06 running acc average = 0.17
End of epoch 2: running error average = 212.59 running acc average = 0.15
End of epoch 3: running error average = 29.14 running acc average = 0.13
End of epoch 4: running error average = 12.91 running acc average = 0.12
End of epoch 5: running error average = 9.02 running acc average = 0.12
End of epoch 6: running error average = 6.01 running acc average = 0.12
End of epoch 7: running error average = 5.11 running acc average = 0.12
End of epoch 8: running error average = 4.50 running acc average = 0.12
End of epoch 9: running error average = 4.02 running acc average = 0.12
End of epoch 10: running error average = 4.06 running acc average = 0.12


In [6]:
# 100 hidden units learning rate 10-5
n_hidden_1 = 100
n_hidden_2 = 100
n_hidden_3 = 100
beta = 0.01 #0.009

weights = {
    'h1': tf.Variable(tf.random_normal([3000, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, 10]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([10]))
}
def multilayer_3hidden100(data):
    layer_1 = tf.nn.relu(tf.matmul(data, weights['h1']) + biases['b1'])
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights['h2']) + biases['b2'])
    layer_3 = tf.nn.relu(tf.matmul(layer_2, weights['h3']) + biases['b3'])
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer
out_layer = multilayer_3hidden100(inputs)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=targets)
error = tf.reduce_mean(per_datapoint_errors)
#regularizer = tf.nn.l2_loss(weights)
#error = tf.reduce_mean(loss + beta * regularization)

per_datapoint_pred_is_correct = tf.equal(tf.argmax(out_layer, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate=0.00001).minimize(error)
sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)
import data_providers as data_providers
train_data = data_providers.MSD10GenreDataProvider('train', batch_size=50, flatten=True, one_hot=True)
valid_data = data_providers.MSD10GenreDataProvider('valid', batch_size=50, flatten=True, one_hot=True)
num_epoch = 50
for e in range(num_epoch):
    running_error = 0.
    running_acc = 0.
    for input_batch, target_batch in train_data:
        _, batch_error, batch_acc = sess.run(
            [train_step, error, accuracy], 
            feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
        running_acc += batch_acc
    running_error /= train_data.num_batches
    running_acc /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e + 1, running_error), 
          'running acc average = {1:.2f}'.format(e + 1, running_acc))

End of epoch 1: running error average = 9899.99 running acc average = 0.14
End of epoch 2: running error average = 5148.98 running acc average = 0.18
End of epoch 3: running error average = 3617.99 running acc average = 0.18
End of epoch 4: running error average = 2626.01 running acc average = 0.19
End of epoch 5: running error average = 1989.34 running acc average = 0.18
End of epoch 6: running error average = 1481.91 running acc average = 0.18
End of epoch 7: running error average = 1063.26 running acc average = 0.19
End of epoch 8: running error average = 742.17 running acc average = 0.18
End of epoch 9: running error average = 497.01 running acc average = 0.17
End of epoch 10: running error average = 344.01 running acc average = 0.17
End of epoch 11: running error average = 268.51 running acc average = 0.16
End of epoch 12: running error average = 245.57 running acc average = 0.15
End of epoch 13: running error average = 227.41 running acc average = 0.15
End of epoch 14: running er

In [8]:
# 200 hidden units learning rate 10-4
n_hidden_1 = 200
n_hidden_2 = 200
n_hidden_3 = 200
beta = 0.01 #0.009

weights = {
    'h1': tf.Variable(tf.random_normal([3000, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, 10]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([10]))
}
def multilayer_3hidden200(data):
    layer_1 = tf.nn.relu(tf.matmul(data, weights['h1']) + biases['b1'])
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights['h2']) + biases['b2'])
    layer_3 = tf.nn.relu(tf.matmul(layer_2, weights['h3']) + biases['b3'])
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer
out_layer = multilayer_3hidden200(inputs)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=targets)
error = tf.reduce_mean(per_datapoint_errors)
#regularizer = tf.nn.l2_loss(weights)
#error = tf.reduce_mean(loss + beta * regularization)

per_datapoint_pred_is_correct = tf.equal(tf.argmax(out_layer, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate=0.0001).minimize(error)
sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)
import data_providers as data_providers
train_data = data_providers.MSD10GenreDataProvider('train', batch_size=50, flatten=True, one_hot=True)
valid_data = data_providers.MSD10GenreDataProvider('valid', batch_size=50, flatten=True, one_hot=True)
num_epoch = 10
for e in range(num_epoch):
    running_error = 0.
    running_acc = 0.
    for input_batch, target_batch in train_data:
        _, batch_error, batch_acc = sess.run(
            [train_step, error, accuracy], 
            feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
        running_acc += batch_acc
    running_error /= train_data.num_batches
    running_acc /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e + 1, running_error), 
          'running acc average = {1:.2f}'.format(e + 1, running_acc))

End of epoch 1: running error average = 5926.74 running acc average = 0.19
End of epoch 2: running error average = 64.26 running acc average = 0.14
End of epoch 3: running error average = 21.80 running acc average = 0.12
End of epoch 4: running error average = 11.23 running acc average = 0.12
End of epoch 5: running error average = 7.85 running acc average = 0.12
End of epoch 6: running error average = 6.42 running acc average = 0.12
End of epoch 7: running error average = 5.28 running acc average = 0.11
End of epoch 8: running error average = 5.11 running acc average = 0.11
End of epoch 9: running error average = 4.42 running acc average = 0.11
End of epoch 10: running error average = 3.88 running acc average = 0.11


In [9]:
# 200 hidden units learning rate 10-5
n_hidden_1 = 200
n_hidden_2 = 200
n_hidden_3 = 200
beta = 0.01 #0.009

weights = {
    'h1': tf.Variable(tf.random_normal([3000, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, 10]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([10]))
}
def multilayer_3hidden200(data):
    layer_1 = tf.nn.relu(tf.matmul(data, weights['h1']) + biases['b1'])
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights['h2']) + biases['b2'])
    layer_3 = tf.nn.relu(tf.matmul(layer_2, weights['h3']) + biases['b3'])
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer
out_layer = multilayer_3hidden200(inputs)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=targets)
error = tf.reduce_mean(per_datapoint_errors)
#regularizer = tf.nn.l2_loss(weights)
#error = tf.reduce_mean(loss + beta * regularization)

per_datapoint_pred_is_correct = tf.equal(tf.argmax(out_layer, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate=0.00001).minimize(error)
sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)
import data_providers as data_providers
train_data = data_providers.MSD10GenreDataProvider('train', batch_size=50, flatten=True, one_hot=True)
valid_data = data_providers.MSD10GenreDataProvider('valid', batch_size=50, flatten=True, one_hot=True)
num_epoch = 10
for e in range(num_epoch):
    running_error = 0.
    running_acc = 0.
    for input_batch, target_batch in train_data:
        _, batch_error, batch_acc = sess.run(
            [train_step, error, accuracy], 
            feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
        running_acc += batch_acc
    running_error /= train_data.num_batches
    running_acc /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e + 1, running_error), 
          'running acc average = {1:.2f}'.format(e + 1, running_acc))

End of epoch 1: running error average = 23589.52 running acc average = 0.18
End of epoch 2: running error average = 11705.73 running acc average = 0.23
End of epoch 3: running error average = 7856.59 running acc average = 0.23
End of epoch 4: running error average = 5513.54 running acc average = 0.23
End of epoch 5: running error average = 3802.50 running acc average = 0.23
End of epoch 6: running error average = 2608.29 running acc average = 0.23
End of epoch 7: running error average = 1848.06 running acc average = 0.22
End of epoch 8: running error average = 1423.08 running acc average = 0.21
End of epoch 9: running error average = 1132.32 running acc average = 0.21
End of epoch 10: running error average = 882.90 running acc average = 0.20


In [2]:
import numpy as np
a = np.load('/afs/inf.ed.ac.uk/user/s17/s1773005/mlpractical/data/inputs.npy')
b = np.load('/afs/inf.ed.ac.uk/user/s17/s1773005/mlpractical/data/targets.npy')
a.shape, b.shape

((34000, 120, 25), (34000,))

In [3]:
import os
os.path.join(
            os.environ['MLP_DATA_DIR'], 'inputs.npy')

'/afs/inf.ed.ac.uk/user/s17/s1773005/mlpractical/data/inputs.npy'

In [18]:
print(np.sum(b == ))

0
